In [ ]:
from niapy.algorithms.basic import (
    BatAlgorithm,
    FireflyAlgorithm,
    ParticleSwarmOptimization,
)
from niapy.problems.ackley import Ackley
import torch
from torch import nn
from PIL import Image
from matplotlib import pyplot as plt
import os
import numpy as np
from tools.ml_tools import get_data_loaders, nn_test, nn_train, LSTM
from util.optimization_data import SingleRunData
from tools.optimization_tools import optimization_runner

from util.constants import (
    RNG_SEED,
    DATASET_PATH,
    BATCH_SIZE,
    EPOCHS,
    POP_SIZE,
    MAX_EVALS,
    NUM_RUNS,
    POP_DIVERSITY_METRICS,
    INDIV_DIVERSITY_METRICS,
)

execute_training = True

### Optimization

In [ ]:
for algorithm in [
    BatAlgorithm(population_size=POP_SIZE),
    FireflyAlgorithm(population_size=POP_SIZE),
    ParticleSwarmOptimization(population_size=POP_SIZE, c1=1.0, c2=1.0),
]:
    problem = Ackley()
    optimization_runner(
        algorithm,
        problem,
        NUM_RUNS,
        DATASET_PATH,
        POP_DIVERSITY_METRICS,
        INDIV_DIVERSITY_METRICS,
        max_evals=MAX_EVALS,
        rng_seed=RNG_SEED,
        parallel_processing=True,
    )

### Metrics comparison

In [ ]:
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        pop_metrics = SingleRunData.import_from_json(run_path).get_pop_diversity_metrics_values(normalize=True)
        pop_metrics.plot(title=" ".join([algorithm, problem]), figsize=(20,5))

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
print("CPUs: ", os.cpu_count())

In [ ]:
train_data_loader, val_data_loader, test_data_loader, labels = get_data_loaders(DATASET_PATH, BATCH_SIZE, problems=["Ackley"], random_state=RNG_SEED)

pop_features, indiv_features, target = next(iter(train_data_loader))
model = LSTM(np.shape(pop_features)[2], np.shape(indiv_features)[1], len(labels))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
model_file_name = f"lstm_model.pt"

if execute_training:
    model.to(device)
    nn_train(model, train_data_loader, val_data_loader, EPOCHS, loss_fn, optimizer, device, model_file_name, verbal=True)
else:
    model = torch.load(model_file_name, map_location=torch.device(device))
    model.to(device)
    if os.path.exists('loss_plot.png'):
        loss_plot = np.asarray(Image.open('loss_plot.png'))
        plt.axis("off")
        plt.imshow(loss_plot)

In [ ]:
nn_test(model, test_data_loader, device, labels=labels, show_classification_report=True)